In [1]:
import sys
import ollama
import prompts
import llm_system
from tqdm import tqdm
sys.path.append('../')
import post_processing

##### Load data

In [2]:
# load the data
data = post_processing.load_data()

##### Instances and parameters

In [3]:
# instances
instances_interpret = [27606, 15179, 29493, 19985, 3144, 19966, 945, 6368, 18524, 30216, 13578, 13400, 5213, 2855, 9869, 15183, 13296, 15463, 19307, 4658]
example_instances = [17170, 570]
# parameteres for the data
granularity = 'hourly'
# parameteres for the ML component
ml_task = 'clustering'
real_task = 'well-being'
target = 'cluster'
target_encoding = {0: 'negative', 1: 'positive'}
# parameteres for the XAI component
scope = 'local'
xai_method = 'lime'

##### LLM parameters

In [4]:
learning = 'few'
model = 'mistral'

##### Create system

In [ ]:
if learning == 'zero':
    system_content = llm_system.create_system(ml_task, granularity, target_encoding, target, real_task, learning, scope, xai_method, str(0))
elif learning == 'one':
    system_content = llm_system.create_system(ml_task, granularity, target_encoding, target, real_task, learning, scope, xai_method, str(1))
elif learning == 'few':
    system_content = llm_system.create_system(ml_task, granularity, target_encoding, target, real_task, learning, scope, xai_method, str(2))
system_content

##### Prompt and get responses

In [6]:
pbar = tqdm(total=len(instances_interpret), desc="Processing instances", unit="inst")
# for each instance
for instance_interpret in instances_interpret:
    # prepare the prompt based on the learning technique
    if learning == 'zero':
        prompt = prompts.zero_prompt(data, instance_interpret, target, target_encoding, granularity, real_task)
    elif learning == 'one':
        prompt = prompts.one_prompt(data, instance_interpret, example_instances[0], target, target_encoding, granularity, real_task)
    elif learning == 'few':
        prompt = prompts.few_prompt(data, instance_interpret, example_instances, target, target_encoding, granularity, real_task)

    # prompt the llm
    response = ollama.chat(model=model, messages=[
    {
        'role': 'system',
        'content': system_content
    },
    {
        'role': 'user',
        'content': prompt,
    },
    ])
    response = response['message']['content']
    response_path = '../data/llms_output/' + learning + '/' + model + '_' + str(instance_interpret) + '.txt'
    with open(response_path, 'w') as f:
        f.write(response)
    pbar.update(1) 
pbar.close()

Processing instances: 100%|██████████| 20/20 [10:55:43<00:00, 1967.18s/inst] 
